## Trained Vowel Classfication From Speech
- created: 13 Apr 2020 </br>
To detect the error in an inversion data because the formant cannot be used to compare the model

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras
from joblib import dump, load
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio
from sklearn.metrics import average_precision_score, precision_score, recall_score, roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tf.random.set_seed(42)

## Function

In [3]:
def prep_data(data_dir,prep_folder):

    features = np.load(join(data_dir,prep_folder,'features.npy'))
    
    with open(join(data_dir,'syllable_name.txt')) as f:
        label = np.array([char for line in f for word in line.split(',') for char in word.strip()])
        
    print('Predict Data {} {}'.format(features.shape, label.shape))
    return features, label

In [4]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
#     print('Confusion Matrix')
    print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

## Load Data 

In [6]:
DATA_PATH = '../../data/d_records/record_total_data/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)

Predict Data (3078, 23, 39) (3078,)


In [7]:
set(label)

{'7', 'E', 'M', 'O', 'a', 'e', 'i', 'o', 'u'}

In [8]:
label.shape

(3078,)

### One-hot label 

In [9]:
label=label.reshape(-1,1)

enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(label)

label_oe = enc.transform(label).toarray()

# dump(enc, 'onehot_pred.joblib')

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label_oe, test_size=0.2)

### Defined (Between) Embedding Model

In [11]:
PHONETIC_OUTPUT = len(['7', 'E', 'M', 'O', 'a', 'e', 'i', 'o', 'u'])
EMBEDDED_FEATURE = 64

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)

def embeded_model(input_shape_1,input_shape_2):
    inputs = keras.Input(shape=(input_shape_1,input_shape_2))
    x = pLSTM(EMBEDDED_FEATURE)(inputs)
    x = Dropout(rate=0.5)(x)
    x = pLSTM(EMBEDDED_FEATURE, return_sequences=False)(x)
    x = Dropout(rate=0.5)(x)
    outputs = Dense(PHONETIC_OUTPUT, activation='softmax', kernel_initializer='he_uniform')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [12]:
embedded = embeded_model(23, 39)
embedded.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23, 39)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 23, 64)            26624     
_________________________________________________________________
dropout (Dropout)            (None, 23, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 9)                 585       
Total params: 60,233
Trainable params: 60,233
Non-trainable params: 0
_________________________________________________________

### Compile Model 

In [148]:
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

### Train Model 

In [14]:
PATIENCE = 15
BATCH_SIZE = 128
EPOCHS = 200

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [150]:
history = embedded.fit(X_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_split=0.2,
    callbacks=callback_list)

Train on 1969 samples, validate on 493 samples
Epoch 1/200
1969/1969 - 4s - loss: 2.1652 - accuracy: 0.2331 - val_loss: 2.0904 - val_accuracy: 0.5132
Epoch 2/200
1969/1969 - 0s - loss: 2.0553 - accuracy: 0.4794 - val_loss: 1.9252 - val_accuracy: 0.6795
Epoch 3/200
1969/1969 - 0s - loss: 1.9155 - accuracy: 0.6369 - val_loss: 1.7628 - val_accuracy: 0.7688
Epoch 4/200
1969/1969 - 0s - loss: 1.7694 - accuracy: 0.7669 - val_loss: 1.6248 - val_accuracy: 0.8641
Epoch 5/200
1969/1969 - 0s - loss: 1.6480 - accuracy: 0.8532 - val_loss: 1.5439 - val_accuracy: 0.8783
Epoch 6/200
1969/1969 - 0s - loss: 1.5547 - accuracy: 0.9035 - val_loss: 1.5035 - val_accuracy: 0.8945
Epoch 7/200
1969/1969 - 0s - loss: 1.5017 - accuracy: 0.9253 - val_loss: 1.4857 - val_accuracy: 0.9026
Epoch 8/200
1969/1969 - 0s - loss: 1.4768 - accuracy: 0.9375 - val_loss: 1.4724 - val_accuracy: 0.9108
Epoch 9/200
1969/1969 - 0s - loss: 1.4576 - accuracy: 0.9436 - val_loss: 1.4667 - val_accuracy: 0.9087
Epoch 10/200
1969/1969 - 0

In [151]:
# embedded.save('pred_vowel.hdf5')

In [153]:
pred = embedded.predict(X_test)
evaluate_model(pred, y_test)

[[71  4  1  1  0  2  0  0  0]
 [ 1 70  0  0  0  1  0  0  0]
 [ 0  0 58  0  0  2  1  0  0]
 [ 0  0  0 73  0  0  0  1  0]
 [ 0  0  0  1 53  0  0  0  0]
 [ 0  0  0  0  0 64  3  0  0]
 [ 0  0  0  0  0  1 66  0  0]
 [ 0  0  0  0  0  0  0 64  1]
 [ 0  0  0  0  0  0  1  2 74]]
Classification Report
              precision    recall  f1-score   support

           7       0.99      0.90      0.94        79
           E       0.95      0.97      0.96        72
           M       0.98      0.95      0.97        61
           O       0.97      0.99      0.98        74
           a       1.00      0.98      0.99        54
           e       0.91      0.96      0.93        67
           i       0.93      0.99      0.96        67
           o       0.96      0.98      0.97        65
           u       0.99      0.96      0.97        77

    accuracy                           0.96       616
   macro avg       0.96      0.96      0.96       616
weighted avg       0.96      0.96      0.96       616



### Train using all data 

In [ ]:
embedded = embeded_model(23, 39)
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
history = embedded.fit(features,label_oe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    callbacks=callback_list)

In [155]:
embedded.save('pred_vowel_all_data.hdf5')

## Load Model

In [20]:
embedded = models.load_model('pred_vowel_all_data.hdf5')
enc = load('onehot_pred.joblib')

## Predict 31
- BiLSTM 

In [21]:
DATA_PATH = '../../data/inversion/inversion_31/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[177  16  16   0   0   5   1   1   0]
 [  6 196   1   0   5   8   0   0   0]
 [  2   1 164   0   0  29  18   0   2]
 [  2   6   2 163  29   0   0  14   0]
 [  1  10   4  12 188   1   0   0   0]
 [  0   2   2   0   0 211   1   0   0]
 [  0   0   6   0   0  10 197   0   3]
 [ 10   2   1   7   0   5   0 145  46]
 [  4   2  20   0   0   5  15  12 158]]
Classification Report
              precision    recall  f1-score   support

           7       0.88      0.82      0.85       216
           E       0.83      0.91      0.87       216
           M       0.76      0.76      0.76       216
           O       0.90      0.75      0.82       216
           a       0.85      0.87      0.86       216
           e       0.77      0.98      0.86       216
           i       0.85      0.91      0.88       216
           o       0.84      0.67      0.75       216
           u       0.76      0.73      0.74       216

    accuracy                           0.82     

In [22]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.8255784132730615 Recall: 0.8225308641975309 AUC: 0.951728252171925


## Predict 86
- BiLSTM without Speaker Simulation

In [23]:
DATA_PATH = '../../data/inversion/inversion_86/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[140  36  18   5   0   6   2   8   1]
 [ 13 161   1   2  15  17   6   1   0]
 [ 12   1 144   0   0  37  18   1   3]
 [  7   5   0 142  39   0   0  22   1]
 [  3  22   2  18 169   2   0   0   0]
 [  7   4   5   0   0 192   8   0   0]
 [  1   0   5   0   0  28 181   0   1]
 [ 17  10   1  11   0   9   0 106  62]
 [ 14   1   9   0   0   9  19  20 144]]
Classification Report
              precision    recall  f1-score   support

           7       0.65      0.65      0.65       216
           E       0.67      0.75      0.71       216
           M       0.78      0.67      0.72       216
           O       0.80      0.66      0.72       216
           a       0.76      0.78      0.77       216
           e       0.64      0.89      0.74       216
           i       0.77      0.84      0.80       216
           o       0.67      0.49      0.57       216
           u       0.68      0.67      0.67       216

    accuracy                           0.71     

In [24]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.7136281438091008 Recall: 0.7093621399176955 AUC: 0.923747154111416


## Predict 87
- BiLSTM without Speaker Simulation and data augmentation

In [25]:
DATA_PATH = '../../data/inversion/inversion_87/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[131  30  23   5   0  17   0   4   6]
 [ 13 138   5   4  34  21   1   0   0]
 [ 23   2 115   0   1  63   8   0   4]
 [  8  11   7 103  37   0   0  43   7]
 [ 15  37  24  25 112   3   0   0   0]
 [ 15  12   9   0   0 175   4   0   1]
 [ 10   0  22   0   0  63 115   0   6]
 [ 22  24   6  14   0   5   1  88  56]
 [ 39   1  11   0   0  17   8  26 114]]
Classification Report
              precision    recall  f1-score   support

           7       0.47      0.61      0.53       216
           E       0.54      0.64      0.59       216
           M       0.52      0.53      0.53       216
           O       0.68      0.48      0.56       216
           a       0.61      0.52      0.56       216
           e       0.48      0.81      0.60       216
           i       0.84      0.53      0.65       216
           o       0.55      0.41      0.47       216
           u       0.59      0.53      0.56       216

    accuracy                           0.56     

In [26]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.5865605592568944 Recall: 0.5612139917695473 AUC: 0.844167059613626


## Predict 88
- BiLSTM without data augmentation

In [27]:
DATA_PATH = '../../data/inversion/inversion_88/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[159  19  10   6   1   9   0   5   7]
 [ 10 175   1   4  10  16   0   0   0]
 [ 17   3 143   0   0  37  13   0   3]
 [  5   3   0 114  46   0   0  37  11]
 [  6  62   4  14 126   4   0   0   0]
 [  5   4   3   0   1 190  13   0   0]
 [  2   0  13   0   0  28 171   0   2]
 [ 12  10   2   7   0   3   0 124  58]
 [ 25   0  29   1   0  16  15  18 112]]
Classification Report
              precision    recall  f1-score   support

           7       0.66      0.74      0.70       216
           E       0.63      0.81      0.71       216
           M       0.70      0.66      0.68       216
           O       0.78      0.53      0.63       216
           a       0.68      0.58      0.63       216
           e       0.63      0.88      0.73       216
           i       0.81      0.79      0.80       216
           o       0.67      0.57      0.62       216
           u       0.58      0.52      0.55       216

    accuracy                           0.68     

In [28]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.6827627682928141 Recall: 0.6759259259259259 AUC: 0.9013021428707515


### Predict Data 10
- LTRCNN

In [29]:
DATA_PATH = '../../data/inversion/inversion_10/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[176  23   8   1   0   6   0   2   0]
 [  8 195   0   1   2  10   0   0   0]
 [ 17   2 154   0   0  23  17   0   3]
 [ 11   4   4 149  31   0   0  15   2]
 [  2  43   7  17 146   1   0   0   0]
 [  5   7   4   0   0 198   2   0   0]
 [  0   0  10   0   0  41 165   0   0]
 [ 29   0   0  17   0   2   0 120  48]
 [ 14   2  36   0   0   4  15  19 126]]
Classification Report
              precision    recall  f1-score   support

           7       0.67      0.81      0.74       216
           E       0.71      0.90      0.79       216
           M       0.69      0.71      0.70       216
           O       0.81      0.69      0.74       216
           a       0.82      0.68      0.74       216
           e       0.69      0.92      0.79       216
           i       0.83      0.76      0.80       216
           o       0.77      0.56      0.65       216
           u       0.70      0.58      0.64       216

    accuracy                           0.74     

In [30]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.7429924713303202 Recall: 0.7350823045267489 AUC: 0.9316162146585886


### Predict Data 65 
- Retrain model Conv_bilstm

In [31]:
DATA_PATH = '../../data/inversion/inversion_65/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[150  23  20   6   1  12   0   4   0]
 [ 14 187   0   2   6   6   0   1   0]
 [ 10   1 160   0   0  24  17   0   4]
 [  4  15   0 134  44   0   0  17   2]
 [  4  31   1  39 138   2   0   1   0]
 [  0   3   1   0   0 199  13   0   0]
 [  0   0   5   0   0  29 181   0   1]
 [  9   1   0   9   3   1   0 133  60]
 [  1   0  10   0   1   3  15  18 168]]
Classification Report
              precision    recall  f1-score   support

           7       0.78      0.69      0.74       216
           E       0.72      0.87      0.78       216
           M       0.81      0.74      0.77       216
           O       0.71      0.62      0.66       216
           a       0.72      0.64      0.67       216
           e       0.72      0.92      0.81       216
           i       0.80      0.84      0.82       216
           o       0.76      0.62      0.68       216
           u       0.71      0.78      0.75       216

    accuracy                           0.75     

In [32]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.7479286425724162 Recall: 0.7458847736625515 AUC: 0.9306701650853528


### Predict Data 30
- Retrain model LSTM

In [33]:
DATA_PATH = '../../data/inversion/inversion_30/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[166  22  10   5   0   7   0   5   1]
 [  7 193   0   1   1  13   1   0   0]
 [ 21   0 154   0   0  21  17   1   2]
 [  4  13   0 125  51   0   0  22   1]
 [  1  45   2  17 148   3   0   0   0]
 [  1   2   5   0   0 205   3   0   0]
 [  2   0  27   0   0  18 169   0   0]
 [ 13   1   3  10   1   5   0 134  49]
 [  4   0  15   1   0   4  19  16 157]]
Classification Report
              precision    recall  f1-score   support

           7       0.76      0.77      0.76       216
           E       0.70      0.89      0.78       216
           M       0.71      0.71      0.71       216
           O       0.79      0.58      0.67       216
           a       0.74      0.69      0.71       216
           e       0.74      0.95      0.83       216
           i       0.81      0.78      0.80       216
           o       0.75      0.62      0.68       216
           u       0.75      0.73      0.74       216

    accuracy                           0.75     

In [34]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.7493894691774862 Recall: 0.7463991769547325 AUC: 0.9181306917771681


### Predict Data 8
- Retrain model FCNN

In [35]:
DATA_PATH = '../../data/inversion/inversion_8/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
[[156  20  11   6   2   9   1   5   6]
 [ 11 116   0   7  68  14   0   0   0]
 [ 17   4 145   0   0  38   8   0   4]
 [ 28  13   0  55 104   1   0  14   1]
 [  1   3   2  13 194   3   0   0   0]
 [ 15  14  11   0   0 163  12   1   0]
 [  0   0  18   0   2 129  64   0   3]
 [ 24   0   0  18   0   1   0  96  77]
 [ 29   0  14   0   1  14   3  13 142]]
Classification Report
              precision    recall  f1-score   support

           7       0.56      0.72      0.63       216
           E       0.68      0.54      0.60       216
           M       0.72      0.67      0.70       216
           O       0.56      0.25      0.35       216
           a       0.52      0.90      0.66       216
           e       0.44      0.75      0.55       216
           i       0.73      0.30      0.42       216
           o       0.74      0.44      0.56       216
           u       0.61      0.66      0.63       216

    accuracy                           0.58     

In [36]:
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))

Precision: 0.617382844666726 Recall: 0.58179012345679 AUC: 0.8760179410055632


## Evaluated Model for Preception Test

In [37]:
DATA_PATH = '../../data/inversion_perception/selected/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))
evaluate_model(pred, label_oe)

Predict Data (162, 23, 39) (162,)
Precision: 0.9534437946718648 Recall: 0.9506172839506173 AUC: 0.9960991083676269
[[17  1  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0]
 [ 0  0 17  0  0  1  0  0  0]
 [ 0  0  0 18  0  0  0  0  0]
 [ 0  0  0  1 17  0  0  0  0]
 [ 0  0  0  0  0 17  1  0  0]
 [ 0  0  0  0  0  0 18  0  0]
 [ 0  0  0  0  0  0  0 15  3]
 [ 0  0  1  0  0  0  0  0 17]]
Classification Report
              precision    recall  f1-score   support

           7       1.00      0.94      0.97        18
           E       0.95      1.00      0.97        18
           M       0.94      0.94      0.94        18
           O       0.95      1.00      0.97        18
           a       1.00      0.94      0.97        18
           e       0.94      0.94      0.94        18
           i       0.95      1.00      0.97        18
           o       1.00      0.83      0.91        18
           u       0.85      0.94      0.89        18

    accuracy                           0.95       

## Monosyllabic Model 37

In [38]:
DATA_PATH = '../../data/inversion/inversion_37/'
PREP_FOLDER = 'prep_data'
features, label = prep_data(DATA_PATH, PREP_FOLDER)
label= label.reshape(-1,1)
label_oe = enc.transform(label).toarray()
pred = embedded.predict(features)
class_pred = np.argmax(pred, axis=1)
class_actual = np.argmax(label_oe, axis=1)
precision = precision_score(class_actual, class_pred, average='macro')
recall = recall_score(class_actual, class_pred, average='macro')
auc = roc_auc_score(label_oe, pred)
print('Precision: {} Recall: {} AUC: {}'.format(precision,recall,auc))
evaluate_model(pred, label_oe)

Predict Data (1944, 23, 39) (1944,)
Precision: 0.773863262630748 Recall: 0.7669753086419754 AUC: 0.9349773995960982
[[157  18  13   5   2  15   0   3   3]
 [  8 184   0   3   5  14   2   0   0]
 [ 15   0 154   0   0  32  14   0   1]
 [  2   5   0 161  20   0   0  27   1]
 [  2  26   6  24 153   5   0   0   0]
 [  2   0   3   0   0 196  15   0   0]
 [  0   0  10   0   0  24 182   0   0]
 [  9   0   2   9   0   8   0 139  49]
 [  3   5  10   2   1   7  13  10 165]]
Classification Report
              precision    recall  f1-score   support

           7       0.79      0.73      0.76       216
           E       0.77      0.85      0.81       216
           M       0.78      0.71      0.74       216
           O       0.79      0.75      0.77       216
           a       0.85      0.71      0.77       216
           e       0.65      0.91      0.76       216
           i       0.81      0.84      0.82       216
           o       0.78      0.64      0.70       216
           u       0.75